In [26]:
import pandas as pd
from datetime import datetime


In [27]:
df = pd.read_csv('activity.csv').to_json(orient='records')
# postal = pd.read_csv('SG_postal.csv')
# activity_df = df.merge(postal, left_on='postalCode', right_on='postal_code', how='left')

In [32]:
def get_demographics_report(data, timeframe='monthly'):
    # Load data
    activity_df = pd.read_json(data)

    zones = pd.DataFrame({
    "Zone": ["City", "City", "South", "South", "West", "City", "City", "Central", "Central", "Central", 
             "Central", "Central", "East", "East", "East", "East", "East", "East", "North", "North",
             "West", "West", "West", "West", "North", "North", "North", "North"],
    "District": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
    "PostalBeginsWith": [
        "01, 02, 03, 04, 05, 06", "07, 08", "14, 15, 16", "09, 10", "11, 12, 13", "17", "18, 19", 
        "20, 21", "22, 23", "24, 25, 26, 27", "28, 29, 30", "31, 32, 33", "34, 35, 36, 37", "38, 39, 40, 41",
        "42, 43, 44, 45", "46, 47, 48", "49, 50, 81", "51, 52", "53, 54, 55, 82", "56, 57", "58, 59", 
        "60, 61, 62, 63, 64", "65, 66, 67, 68", "69, 70, 71", "72, 73", "77, 78", "75, 76", "79, 80"
    ]})
    # Extract the first two digits for mapping
    zones['PostalBeginsWith'] = zones['PostalBeginsWith'].apply(lambda x: [y[:2] for y in x.split(', ')])
    zones_exploded = zones.explode('PostalBeginsWith')

    activity_df['startTime'] = pd.to_datetime(activity_df['startTime'])
    activity_df['day'] = activity_df['startTime'].dt.day
    activity_df['year'] = activity_df['startTime'].dt.year
    activity_df['month'] = activity_df['startTime'].dt.month
    activity_df['quarter'] = activity_df['startTime'].dt.quarter
    activity_df['postalCodeFirstTwo'] = activity_df['postalCode'].astype(str).str.zfill(6).str[:2]
    activity_df = activity_df.merge(zones_exploded, how='left', left_on='postalCodeFirstTwo', right_on='PostalBeginsWith')
    activity_df['rowCount'] = 1


    # Convert boolean columns to integer type
    activity_df['drivingLicence'] = activity_df['drivingLicence'].astype(int)
    activity_df['pwdTrained'] = activity_df['pwdTrained'].astype(int)

    # Get dummies for categorical columns
    attendance_dummies = pd.get_dummies(activity_df['attendanceStatus'], prefix='attendance')
    tags_dummies = activity_df['tags'].str.get_dummies(sep=',')
    citizenship_dummies = pd.get_dummies(activity_df['citizenshipType'], prefix='citizenship')
    employment_dummies = pd.get_dummies(activity_df['employmentStatus'], prefix='employment')
    gender_dummies = pd.get_dummies(activity_df['gender'], prefix='gender')
    zone_dummies = pd.get_dummies(activity_df['Zone'], prefix='zone')

    current_year = pd.Timestamp('now').year
    activity_df['dateOfBirth'] = pd.to_datetime(activity_df['dateOfBirth'])
    activity_df['age'] = current_year - activity_df['dateOfBirth'].dt.year

    # Group age into categories and get dummies
    bins = [0, 13, 21, 50, 65, float('inf')]
    labels = ['under13', 'under21', 'under50', 'under65', 'over65']
    activity_df['ageGroup'] = pd.cut(activity_df['age'], bins=bins, labels=labels, right=False)
    age_group_dummies = pd.get_dummies(activity_df['ageGroup'])

    # Concatenate all dummies and the original DataFrame (excluding original categorical columns)
    final_df = pd.concat([
        activity_df.drop(['attendanceStatus','tags', 'citizenshipType', 'employmentStatus', 'gender', 'age', 'ageGroup', 'Zone'], axis=1),
        attendance_dummies, tags_dummies, citizenship_dummies, employment_dummies, gender_dummies, age_group_dummies, zone_dummies
    ], axis=1)

    # Group by year and month, and sum the numHours and other one-hot encoded columns
    if timeframe == 'annual':
        final_df = final_df.groupby(['year'])
    elif timeframe == 'quarterly':
        final_df = final_df.groupby(['year', 'quarter'])
    elif timeframe == 'monthly':
        final_df = final_df.groupby(['year', 'month'])
    elif timeframe == 'daily':
        final_df = final_df.groupby(['year', 'month', 'day'])
    else:
        raise ValueError('Invalid timeframe')

    summary = final_df.agg({
        'rowCount': 'count', #count number of records
        'numHours': 'sum',  # Sum numHours for total hours
        **{col: 'sum' for col in attendance_dummies.columns},  # Sum for each attendance status
        **{col: 'sum' for col in tags_dummies.columns},  # Sum for each tag column
        'drivingLicence': 'sum',
        'pwdTrained': 'sum',
        **{col: 'sum' for col in citizenship_dummies.columns},  # Sum for each citizenship status
        **{col: 'sum' for col in employment_dummies.columns},  # Sum for each employment status
        **{col: 'sum' for col in gender_dummies.columns},  # Sum for each gender
        **{col: 'sum' for col in age_group_dummies.columns},  # Sum for each age group
        **{col: 'sum' for col in zone_dummies.columns},  # Sum for each zone
    }).reset_index()

    return summary#.to_json(orient='records')

In [35]:
output = get_demographics_report(df, 'monthly')

In [13]:
import pandas as pd

def get_attendance_report(user_ids, data, timeframe='monthly'):
    # Load data
    activity_df = pd.read_json(data)

    # Filter DataFrame for the given user IDs
    activity_df = activity_df[activity_df['userId'].isin(user_ids)]
    
    user_details_columns = ['userId', 'fullName', 'employmentStatus', 'contactNumber', 'gender', 
                            'occupation', 'skills', 'declarations', 'drivingLicence', 'address', 
                            'pwdTrained', 'dateOfBirth']
    user_details_df = activity_df[user_details_columns].drop_duplicates(subset=['userId'])
    # Calculate age from dateOfBirth
    current_year = datetime.now().year
    user_details_df['dateOfBirth'] = pd.to_datetime(user_details_df['dateOfBirth'])
    user_details_df['age'] = current_year - user_details_df['dateOfBirth'].dt.year

    # Ensure startTime is in datetime format
    activity_df['startTime'] = pd.to_datetime(activity_df['startTime'])
    activity_df['year'] = activity_df['startTime'].dt.year
    activity_df['month'] = activity_df['startTime'].dt.month
    activity_df['quarter'] = activity_df['startTime'].dt.quarter
    activity_df['day'] = activity_df['startTime'].dt.day

    # Convert attendanceStatus to dummies
    attendance_status_dummies = pd.get_dummies(activity_df['attendanceStatus'], prefix='attendanceStatus')

    # Get dummies for tags
    tags_dummies = activity_df['tags'].str.get_dummies(sep=',')

    # Add a row count column for counting the number of events signed up
    activity_df['eventSignUpCount'] = 1

    # Concatenate dummies and the original DataFrame
    activity_df = pd.concat([
        activity_df[['userId', 'year', 'month', 'quarter', 'day', 'numHours', 'eventSignUpCount']],
        attendance_status_dummies,
        tags_dummies
    ], axis=1)

    # Define group by columns based on timeframe
    group_by_columns = ['userId', 'year', 'month', 'quarter', 'day']
    if timeframe == 'annual':
        group_by_columns = ['userId', 'year']
    elif timeframe == 'quarterly':
        group_by_columns = ['userId', 'year', 'quarter']
    elif timeframe == 'monthly':
        group_by_columns = ['userId', 'year', 'month']
    elif timeframe == 'daily':
        group_by_columns = ['userId', 'year', 'month', 'day']
    else:
        raise ValueError('Invalid timeframe specified')

    # Group by and summarize
    grouped_df = activity_df.groupby(group_by_columns).agg({
        'eventSignUpCount': 'sum',
        'numHours': 'sum',
        **{col: 'sum' for col in attendance_status_dummies.columns},
        **{col: 'sum' for col in tags_dummies.columns},
    }).reset_index()

    return grouped_df, user_details_df


In [37]:
example_users = ["65c1c5f0919469d96a250701", "65c1c5a1919469d96a24fdb5", "65c1c5fc919469d96a25088d", "65c1c5d7919469d96a2503f9"]
output2 = get_attendance_report(example_users, df, 'daily')[0]
output3 = get_attendance_report(example_users, df, 'daily')[1]
output2.to_csv('attendance_report.csv', index=False)
output3.to_csv('user_details.csv', index=False)



,userId,fullName,employmentStatus,contactNumber,gender,occupation,skills,declarations,drivingLicence,address,pwdTrained,dateOfBirth,age
8,65c1c5f0919469d96a250701,Cornelius Metz-Kohler,Student,90059,Female,Corporate Data Analyst,innocent versus oh butler paragraph,Ager pauper autem currus vivo eius asperiores ...,False,7688 Prosacco Mission,False,2000-02-25 11:36:59.732000+00:00,24
16,65c1c5fc919469d96a25088d,Mallory Gleason,Student,23808,Other,National Program Engineer,yowza that aha quickly couch,Utique coerceo avarus alius collum vitiosus ad...,False,215 Maple Avenue,True,2017-01-29 14:20:08.036000+00:00,7
42,65c1c5d7919469d96a2503f9,Keyshawn Friesen,Employed / Self- Employed,72103,Other,Principal Metrics Executive,anenst eventually modulo scheme likewise,Victus statim vulariter sublime supra dolorem ...,True,20917 Grange Avenue,True,2004-12-13 17:43:55.279000+00:00,20
162,65c1c5a1919469d96a24fdb5,Neoma Connelly,Student,72318,Female,Chief Directives Assistant,adsorb apparel sick the easy,Tertius degero vilitas bonus conculco aptus.,True,7403 Rosalind Station,False,2005-10-15 22:52:12.813000+00:00,19


In [7]:
#pip install folium
import folium
from folium.plugins import HeatMap
import pandas as pd

df = activity_df
# Create a map centered around the average latitude and longitude
map_center_lat = df['lat'].mean()
map_center_lon = df['lon'].mean()
volunteer_map = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12)

# Aggregate hours by location
location_hours = df.groupby(['lat', 'lon'])['numHours'].sum().reset_index()

# Create a list of lists where each inner list contains the latitude, longitude, and weight (hours)
heat_data = [[row['lat'], row['lon'], row['numHours']] for index, row in location_hours.iterrows()]

# Create a heat map layer and add it to the map
HeatMap(heat_data).add_to(volunteer_map)

# Save the map to an HTML file
volunteer_map.save('volunteer_hours_distribution_map.html')

# Display the map in a Jupyter notebook (if you're using one, otherwise the map will be saved to the HTML file)
volunteer_map


In [36]:
# Save to CSV
output.to_csv('monthly_summary.csv', index=False)

In [9]:
import pandas as pd
import json
from numpy import int64
monthly_summary = pd.read_csv('monthly_summary.csv')
json_data = monthly_summary.to_json(orient='records')



In [18]:


# Function to select a row based on year and month and format the output with grouped categories
def get_month_summary_grouped(data, year, month):
    # Convert the JSON data to a dataframe
    df = pd.DataFrame(json.loads(data))
    
    # Select the row for the given year and month
    selected_row = df[(df['year'] == year) & (df['month'] == month)]
    
    # If no matching row found, return a message
    if selected_row.empty:
        return "No data found for the specified year and month."
    
    # Convert numpy.int64 to int for JSON serialization
    def convert_int64(val):
        return int(val) if isinstance(val, int64) else val

    # Dictionary to hold the grouped categories
    grouped_categories = {
        'Year': convert_int64(selected_row['year'].values[0]),
        'Month': convert_int64(selected_row['month'].values[0]),
        'Number of Activities': convert_int64(selected_row['rowCount'].values[0]),
        'Total Hours': convert_int64(selected_row['numHours'].values[0]),
        'Skills': selected_row[['drivingLicence', 'pwdTrained']].applymap(convert_int64).to_dict('records')[0],
        'Attendance': selected_row.filter(regex='attendance_').applymap(convert_int64).to_dict('records')[0],
        'Theme Prevalence': selected_row.iloc[:, 7:33].applymap(convert_int64).to_dict('records')[0],
        'Citizenship': selected_row.filter(regex='citizenship_').applymap(convert_int64).to_dict('records')[0],
        'Employment': selected_row.filter(regex='employment_').applymap(convert_int64).to_dict('records')[0],
        'Gender': selected_row.filter(regex='gender_').applymap(convert_int64).to_dict('records')[0],
        'AgeGroups': selected_row.filter(regex='^under|^over').applymap(convert_int64).to_dict('records')[0],
        'Zones': selected_row.filter(regex='zone_').applymap(convert_int64).to_dict('records')[0]
    }
    
    # Convert the grouped categories dictionary to JSON
    return grouped_categories

# Example usage:
# Assume we want to get the summary for March 2022
example_year = 2022
example_month = 3
output = get_month_summary_grouped(json_data, example_year, example_month)